# Labeling with GPT4o

This notebook labels the samples using the GPT4o model. 

In [27]:
MODEL_OUTPUT = 'gpt4o_outputs_v2'
SPAN_ANNOTATION = 'span_annotations_v2'

In [28]:
import os 
from openai import OpenAI

KEY = os.environ['OPENAI_KEY']

Given the length of the samples could affect the labeling performance, I computed the token length of the samples using [Open AI online tokenizer](https://platform.openai.com/tokenizer) and conducted a literature review to find the optimal length of the samples for the GPT4o model, the related papers are listed below. It seems to me our prompt and samples are short enough to be processed in batches. 

BABILong: a long-context needle-in-a-haystack benchmark for LLMs

https://github.com/booydar/babilong

https://arxiv.org/pdf/2404.02060

https://github.com/TIGER-AI-Lab/LongICLBench



In [29]:
system_message = """You are a crowd-source worker who identifies and label hate speech in Turkish tweets. For each tweet, find the hate speech spans targeting the given group and label them with the hate speech category.
Hate Speech Categories:
- Exclusive/Discriminatory Discourse
- Exaggeration; Generalization; Attribution; Distortion 
- Threat of Enmity; War; Attack; Murder; or Harm 
- Symbolization 
- Swearing; Insult; Defamation; Dehumanization
Return a JSON for each tweet:
{ 
	"tweet": {
		"id": "",
		"span": [ 
			{
			"text_span": "",
			"hate_speech_category": ""
			},
			{
			"text_span": "",
			"hate_speech_category": ""
			}
		] 
	}
}
"""

def query_gpt(text):

	
	client = OpenAI(
	    # This is the default and can be omitted
	    api_key=KEY,
	)

	response = client.chat.completions.create(
	    messages=[
	    	{
	    		"role": "system",
	    		"content": system_message,
			},
	        {
	            "role": "user",
	            "content": text,
	        }
	    ],
	    model="gpt-4o-2024-08-06",
	)
	return response.choices[0].message.content


# tweet = '''"
# Tweet id: 1, Target: Kurdish, Text: @shemhamfhorash @aIexstrasza_ @xdomerxd @pusholder Gönül isterdi ki Sinan Oğan kazansın ya da c*p hdpkk safında durmasın, %10 kürt oyu toplayayım diye darmaduman oldular. Beter olsunlar, bunların karşısında bırak RTE'yi taş olsa taşa basarım.\nhttps://t.co/3hHhDqODrr
# Tweet id: 2, Target: Refugee, Text:  Yurt önünde türk yok türk... içeriye anonsla alan bile suriyeli. Bu ne biçim iş. #ülkemdemülteciistemiyorum
# Tweet id: 3, Target: Greek, Text: @username Yunanistan'ın ne kadar borcu var? 1 trilyon dolar? 😂 Umarım savaş başlar ve Türk ordusunun acı gücünü dedelerinizin izmirde tattığı gibi sizde tadarsınız!!! Bu kadar klavye konuşması yeterli. Umarım savaş meydanında karşılaşırız yorgo 😂
# Tweet id: 4, Target: Jew, Text: Filistin-Mısır sınırına itrail saldırı düzenlendi orada da müslümanlar sınıra dayandı diye onlara da saldırmaya başladı kuduz köpek.\n#palestine_egypt\n#filistin_misir https://t.co/r46clGBMs9
# Tweet id: 5, Target: Arab, Text:  @FbVit44 @emredem03 @LuckyBesta Arap sevici yallah çölüne burası Türkiye ve ne mutlu türküm diyeceğiz yallah sen
# Tweet id: 6, Target: Jew, Text: Siz İsrail oğulları hiç mi geçmişinizden ders almazsınız ebabil kuşları nasıl bir fil ordusun yok ettiyse sizinde sonunuzda onların ki gibi olacak !\n\n    #AksadaBaskinVar
# Tweet id: 7, Target: Greek, Text: Yunanistan provokasyon Rumlar kışkırtma peşinde https://t.co/qtUy24gmxq @username aracılığıyla
# Tweet id: 8, Target: Jew, Text: Yorumlarda hintliler " İsrail askeri olarak savşmak istiyorum" yazmışlar. Nedir bu hintlilerdeki itrail aşkı? https://t.co/aO1F2ks4lv
# Tweet id: 9, Target: Refugee, Text: Ya 2-3 gündür denk geliyorum şu suriyeli mevzusunu savunanların çoğunun kadın olması... Yapmayın ablalar yapmayın. Çıkın o elit semtlerinizden suriyeliler afganlar bildiğiniz gibi değil https://t.co/SNjWunjZCe	
# Tweet id: 10, Target: Greek, Text: @Ahaber Dünya Tarihinin En Azametli Tarihini Yazar Şanlı Ecdadimizin Adını Destursuz anan serseriler\n\n""Bu milletin Ferdi Değillerdir""\n\nYoksa kimse Kendi Arasına Sövecek Kadar Aşağılıklaşamaz \n\nBunlar Olsa olsa Yunan Dölü Olması Lazm getir... 
# '''
# output = query_gpt(tweet)

In [51]:
import pandas as pd

samples = pd.read_csv('../data/hateful_tweets_majority_voting_121024.csv')
samples

,Unnamed: 0,hedef_grup,derece,kategori,saldırgan_dil,tweet_text,konu,tweet_id,dil,majority_decision,tweet_text_clean
0,0,['Ülke/Milliyet'],4,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Zayıf,#Katilİsrail https://t.co/A2uX5Uygxs,Isr-Pal,1353048938450976770,NaN,Yahudi Karşıtı,#Katilİsrail
1,3,['Ülke/Milliyet'],6,"['Simgeleştirme', 'Düşmanlık; Savaş; Saldırı; ...",Yok,#Katilİsrail https://t.co/IZL3a33jCI,Isr-Pal,1353049567248457729,Türkçe,Yahudi Karşıtı,#Katilİsrail
2,6,"['Irk/Etnik Köken', 'Din', 'Hedef grup birden ...",5,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Zayıf,@username @username @username Kur’anda Tevratt...,Isr-Pal,1353611898026156032,Türkçe,Yahudi Karşıtı,Kur’anda Tevratta derki itrail oğulları ask...
3,9,['Ülke/Milliyet'],3,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Zayıf,"@veliagbaba Suriyenin iç işlerine karışma, ora...",Isr-Pal,1355236074201751554,Türkçe,Yahudi Karşıtı,"Suriyenin iç işlerine karışma, orada konuşlan..."
4,12,['Belli Görüş/Statü/Uygulama; Mesleki Pozisyon...,0,['Nefret söylemi bulunmuyor'],Yok,"Akşam haberlerinde ""ABD DOĞU AKDENİZ İNİYOR"" b...",Isr-Pal,1356275317271371776,Türkçe,Yahudi Karşıtı,"Akşam haberlerinde ""ABD DOĞU AKDENİZ İNİYOR"" b..."
...,...,...,...,...,...,...,...,...,...,...,...
5033,16711,"['Country/Nationality', 'Specific Opinion/Stat...",1,['Exclusive/Discriminatory Discourse'],NaN,https://ye-now.com/news85294063.html…\nاول ايا...,Arabic,1660028211122561024,NaN,Anti-Immigrant/Refugee,\nاول ايام ذو القعدة الكابوس للمغتربين العاملي...
5034,16714,['Country/Nationality'],4,"['Exclusive/Discriminatory Discourse', 'Symbol...",High,العنوسه عندهم ٦٠٪\nال٤٠٪ متزوجين بنغال وهنود و...,Arabic,1660030789122228225,NaN,Anti-Immigrant/Refugee,العنوسه عندهم ٦٠٪\nال٤٠٪ متزوجين بنغال وهنود و...
5035,16717,"['Race/Ethnicity', 'Country/Nationality', 'Gen...",6,['Exaggeration; Generalization; Attribution; D...,High,وبعد هالصوره اثبات تسويقهم صار فاشل انكشفو الم...,Arabic,1660031044723134464,NaN,Anti-Immigrant/Refugee,وبعد هالصوره اثبات تسويقهم صار فاشل انكشفو الم...
5036,16720,"['Race/Ethnicity', 'Country/Nationality', 'Tar...",6,['Exaggeration; Generalization; Attribution; D...,High,الجنسية اليمنية\nأكثر جنسية تم ضبطها\nفي الحمل...,Arabic,1660070744703868928,NaN,Anti-Immigrant/Refugee,الجنسية اليمنية\nأكثر جنسية تم ضبطها\nفي الحمل...


In [52]:
samples['majority_decision'].value_counts()

majority_decision
Göçmen/Mülteci Karşıtı    1259
Yahudi Karşıtı            1200
Yunan Karşıtı             1121
Ermeni Karşıtı             635
Arap Karşıtı               338
Anti-Immigrant/Refugee     245
Alevi Karşıtı              127
Kürt Karşıtı                64
LGBTI+ Karşıtı              49
Name: count, dtype: int64

In [53]:
translations = {
    'Göçmen/Mülteci Karşıtı': 'Immigrant/Refugee',
    'Yunan Karşıtı': 'Greek',
    'Yahudi Karşıtı': 'Jew',
    'Ermeni Karşıtı': 'Armenian',
    'Arap Karşıtı': 'Arab',
    'LGBTI+ Karşıtı': 'LGBTI+',
    'Kürt Karşıtı': 'Kurdish',
    'Alevi Karşıtı': 'Alevi'
}

samples['target'] = samples['majority_decision'].map(translations)

In [54]:
targets = samples['target'].unique()
targets

array(['Jew', 'Greek', 'Immigrant/Refugee', nan, 'Alevi', 'Arab',
       'Armenian', 'Kurdish', 'LGBTI+'], dtype=object)

In [58]:
samples['model_input'] = samples.apply(lambda x: f"Tweet id: {x['tweet_id']}, Target: {x['target']}, Text: {x['tweet_text']}", axis=1)

In [59]:
samples = samples.dropna(subset=['target'])
samples['target'].value_counts()

target
Immigrant/Refugee    1259
Jew                  1200
Greek                1121
Armenian              635
Arab                  338
Alevi                 127
Kurdish                64
LGBTI+                 49
Name: count, dtype: int64

In [60]:
print(samples[samples['target'] == 'LGBTI+']['model_input'].str.cat(sep='\n'))

Tweet id: 1657927550855773952, Target: LGBTI+, Text: Beni rahatsız eden her işlerinde bir şeytanlık var, halkı düşünüyormuş gibi yapmaları insanların özel hayatlarını deşifre etmesi ama kendilerinin sapık ve bir çoğunun biseksüel olması,bunların hepsinin hainlikte rızası var
Tweet id: 1658005454822285056, Target: LGBTI+, Text: @bernalacin35 Defol şeytan CHP pkk lgbt lilere versinler şu sapık zihniyete bak sen geberdiğinde görürsün kim hak ediyor kıyametin kopacak artık son pişmanlık fayda etmez yol yekınken tövbe et rabbine sarıl dinsiz imansız gitme öbür tarafa
Tweet id: 1658005670178725888, Target: LGBTI+, Text: @MAturkce Duygu sümürüsü para etmedi. 6 milyon oy 4 milyona indi. Sadece Êvdocularin; sapık karisiz kocasiz, LGBT'li olmak, devletsiz, kimliksiz, dünyayı deyiştime solculuk fikirlerinin kürde dayatması  yüzünde daha çok küdü kaybedecektir. Kürdü çantada keklik  gören aptallar bitecek....
Tweet id: 1658015459822690048, Target: LGBTI+, Text: @AlparslanCansiz @nevsinmengu Türk k

In [61]:
targets = ['Arab', 'Armenian', 'Greek', 'Jew', 'Immigrant/Refugee', 'Alevi', 'Kurdish',  'LGBTI+' ] 

In [62]:
from tqdm import tqdm
for target in targets:
    df = samples[samples['target'] == target]
    for input, tweet_id in tqdm(zip(df['model_input'].to_list(), df['tweet_id'].to_list())):
        try: 
            output = query_gpt(input)
            with open(f'../data/{MODEL_OUTPUT}/{target.replace("/", "-")}_{tweet_id}.json', 'w', encoding='utf-8') as f:
                f.write(output.strip('`').replace('json', ''))
        except: 
            pass 

In [38]:
# dictionary to tweet id to text
id2text = samples.set_index('tweet_id')['tweet_text'].to_dict()

In [63]:
from pathlib import Path    
import json 

OUTPUT_PATH = Path(f'../data/{MODEL_OUTPUT}')

for target in targets:
    tweets = []
    for file in OUTPUT_PATH.iterdir():
        if target in file.name: 
            with open((OUTPUT_PATH / file.name.replace("/", "-")), 'r', encoding='utf-8') as f:
                data = json.load(f)
                try:
                    data['tweet']['text'] = id2text[int(data['tweet']['id'])]
                except KeyError:
                    print(f"Tweet id {tweet_id} not found")
                tweets.append(data)     
    with open(f'../data/{SPAN_ANNOTATION}/{target.replace("/", "-")}.json', 'w', encoding='utf-8') as f:
        json.dump(tweets, f, ensure_ascii=False, indent=4)

In [64]:
import json 
import pandas as pd 

for target in targets:
    records = []
    with open(f'../data/{SPAN_ANNOTATION}/{target.replace("/", "-")}.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
        for instance in data:
            try:
                tweet_id = instance['tweet']['id']
                text = instance['tweet']['text']
                spans = instance['tweet']['span']
                span_txt = ",".join([f"{span['text_span']}" for span in spans])
                category_txt = ",".join([f"{span['hate_speech_category']}" for span in spans])
                records.append([tweet_id, target, text, span_txt, category_txt])
            except KeyError:
                print(f"Tweet id {tweet_id} not found")
    df = pd.DataFrame(records, columns=['tweet_id', 'target', 'text', 'span', 'category'])
    df.to_csv(f'../data/{SPAN_ANNOTATION}/{target.replace("/", "-")}.csv', index=False)